# PDF ➜ Nougat Markdown ➜ Chroma (LangChain) RAG — cleaned + annotated

This notebook has **two environment sections**:

1) **`nougat` env**: run Nougat on PDFs, detect skipped pages, and build **merged Markdown** outputs.
2) **`rag` env**: ingest the merged Markdown into a local **Chroma** vector DB using **LangChain + sentence-transformers**.

Notes:
- Run cells **top-to-bottom**.
- If you re-run ingestion, you can either **clear** the Chroma directory or **upsert** with stable IDs (this notebook uses stable IDs).
- Paths below assume your Linux VM home directory; adjust `PDF_DIR`, `NOUGAT_OUT`, `MERGED_OUT`, and `CHROMA_DIR` as needed.


## Environment 1: `nougat`

Activate your Nougat environment (example):

```bash
conda activate nougat
```

Then run the cells in this section to:
- discover PDFs
- run Nougat
- capture and parse skipped-page logs
- generate merged Markdown outputs (Nougat text + fallback text for skipped pages)


In [2]:
from pathlib import Path

PDF_DIR = Path("/home/zappalaj/pdfs")
NOUGAT_OUT = Path("/home/zappalaj/nougat_md")          # nougat raw outputs
MERGED_OUT = Path("/home/zappalaj/nougat_merged_md")   # final merged outputs

NOUGAT_OUT.mkdir(parents=True, exist_ok=True)
MERGED_OUT.mkdir(parents=True, exist_ok=True)

pdfs = sorted(PDF_DIR.glob("*.pdf"))
print("PDFs found:", len(pdfs))
for p in pdfs[:5]:
    print(" -", p.name)


PDFs found: 2
 - J Adv Model Earth Syst - 2020 - Delworth - SPEAR The Next Generation GFDL Modeling System for Seasonal to Multidecadal-1.pdf
 - J Adv Model Earth Syst - 2020 - Lu - GFDL s SPEAR Seasonal Prediction System Initialization and Ocean Tendency Adjustment -1.pdf


Run Nougat and capture logs (including skipped pages)

In [7]:
import subprocess, json, re
from tqdm import tqdm

def run_nougat(pdf_path: Path, out_dir: Path) -> dict:
    cmd = ["nougat", str(pdf_path), "-o", str(out_dir)]
    p = subprocess.run(cmd, text=True, capture_output=True)
    return {
        "pdf": str(pdf_path),
        "returncode": p.returncode,
        "stdout": p.stdout,
        "stderr": p.stderr,
        "cmd": cmd,
    }

runs = []
for pdf in tqdm(pdfs, desc="Running Nougat"):
    r = run_nougat(pdf, NOUGAT_OUT)
    runs.append(r)

# quick summary
ok = sum(r["returncode"] == 0 for r in runs)
bad = len(runs) - ok
print(f"Nougat runs: ok={ok}, failed={bad}")
if bad:
    for r in runs:
        if r["returncode"] != 0:
            print("FAILED:", Path(r["pdf"]).name)
            print(r["stderr"][-1000:])


Running Nougat: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [10:08<00:00, 304.32s/it]

Nougat runs: ok=2, failed=0


Parse skipped pages from Nougat logs

In [9]:
skip_re = re.compile(r"Skipping page\s+(\d+)\s+due to repetitions", re.IGNORECASE)

skipped_pages = {}
for r in runs:
    pdf = Path(r["pdf"])
    pages = [int(m.group(1)) for m in skip_re.finditer(r["stderr"])]
    skipped_pages[pdf] = sorted(set(pages))

# show results
for pdf, pages in skipped_pages.items():
    if pages:
        print(pdf.name, "skipped pages:", pages)


J Adv Model Earth Syst - 2020 - Delworth - SPEAR The Next Generation GFDL Modeling System for Seasonal to Multidecadal-1.pdf skipped pages: [33]
J Adv Model Earth Syst - 2020 - Lu - GFDL s SPEAR Seasonal Prediction System Initialization and Ocean Tendency Adjustment -1.pdf skipped pages: [15]


Extract text for specific pages only

In [10]:
from pypdf import PdfReader

def extract_pages_text(pdf_path: Path, page_numbers_1based: list[int]) -> dict[int, str]:
    """
    Returns {page_number_1based: extracted_text}
    """
    reader = PdfReader(str(pdf_path))
    out = {}
    for p1 in page_numbers_1based:
        idx = p1 - 1  # convert to 0-based
        if 0 <= idx < len(reader.pages):
            text = reader.pages[idx].extract_text() or ""
            out[p1] = text.strip()
        else:
            out[p1] = ""
    return out


Merge and write outputs (ensuring JSON files were made correctly)

In [14]:
import datetime, json
from pathlib import Path
from tqdm.auto import tqdm

# Assumes these already exist from earlier cells:
# - pdfs: list[Path]
# - NOUGAT_OUT: Path
# - MERGED_OUT: Path
# - skipped_pages: dict[Path, list[int]]
# - extract_pages_text(pdf_path, skipped_pages_list) -> dict[int, str]

def find_nougat_output_for_pdf(pdf_path: Path, nougat_out_dir: Path) -> Path | None:
    """
    Returns the nougat output corresponding to this pdf, based ONLY on the pdf stem.
    This prevents cross-wiring (Delworth pdf pointing to Lu mmd, etc.).
    """
    # Most common nougat outputs
    for ext in (".mmd", ".md", ".txt"):
        p = nougat_out_dir / f"{pdf_path.stem}{ext}"
        if p.exists():
            return p

    # If nougat writes into subfolders, search but ONLY accept matches containing the pdf stem.
    stem = pdf_path.stem.lower()
    candidates = []
    for ext in (".mmd", ".md", ".txt"):
        candidates.extend(nougat_out_dir.rglob(f"*{ext}"))

    matches = [p for p in candidates if stem in p.stem.lower()]
    if not matches:
        return None

    # choose newest among the matches
    return max(matches, key=lambda p: p.stat().st_mtime)


def merge_with_fallback(pdf_path: Path, nougat_md_path: Path | None, skipped: list[int]) -> tuple[str, dict]:
    # load nougat text
    if nougat_md_path and nougat_md_path.exists():
        nougat_text = nougat_md_path.read_text(encoding="utf-8", errors="ignore").strip()
    else:
        nougat_text = ""

    fallback = extract_pages_text(pdf_path, skipped) if skipped else {}

    parts = []
    if nougat_text:
        parts.append(nougat_text)
    else:
        parts.append(f"# {pdf_path.name}\n\n*(No Nougat output found; using fallback only.)*\n")

    if skipped:
        parts.append("\n\n---\n\n## Fallback extracted text for skipped pages\n")
        for p1 in skipped:
            txt = fallback.get(p1, "")
            if not txt:
                txt = "(No text could be extracted from this page via fallback.)"
            parts.append(f"\n\n### Page {p1}\n\n{txt}\n")

    merged_text = "\n".join(parts)

    meta = {
        "pdf": str(pdf_path),
        "nougat_output": str(nougat_md_path) if nougat_md_path else None,
        "skipped_pages": skipped,
        "created_utc": datetime.datetime.utcnow().isoformat() + "Z",
        "note": "Nougat markdown plus pypdf fallback for pages Nougat skipped due to repetitions.",
    }
    return merged_text, meta


# ------------------------------------------------------------
# Merge + write outputs with a sanity check
# ------------------------------------------------------------
MERGED_OUT.mkdir(parents=True, exist_ok=True)

written = []
bad = []

for pdf in tqdm(pdfs, desc="Merging outputs"):
    nougat_md = find_nougat_output_for_pdf(pdf, NOUGAT_OUT)
    skipped = skipped_pages.get(pdf, [])
    merged_text, meta = merge_with_fallback(pdf, nougat_md, skipped)

    # sanity check: if we found a nougat output, it should correspond to this pdf
    if nougat_md is not None and pdf.stem.lower() not in nougat_md.stem.lower():
        bad.append((pdf.name, str(nougat_md)))
        # hard fail would be: raise RuntimeError(...)
        # but we’ll just record it and continue

    out_md = MERGED_OUT / (pdf.stem + ".md")
    out_json = MERGED_OUT / (pdf.stem + ".json")

    out_md.write_text(merged_text, encoding="utf-8")
    out_json.write_text(json.dumps(meta, indent=2), encoding="utf-8")

    written.append((pdf.name, out_md, out_json, skipped))

print("Wrote:", len(written), "merged files into", MERGED_OUT)
for name, mdp, jsp, skipped in written[:5]:
    print("-", name, "skipped:", skipped, "->", mdp.name)

if bad:
    print("\nWARNING: Some nougat outputs did not match the PDF stem (possible naming mismatch):")
    for pdf_name, nougat_path in bad[:10]:
        print(" -", pdf_name, "->", nougat_path)
    if len(bad) > 10:
        print(" (and", len(bad) - 10, "more)")


/home/zappalaj/miniconda3/envs/nougat/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Merging outputs: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 18.94it/s]

Wrote: 2 merged files into /home/zappalaj/nougat_merged_md
- J Adv Model Earth Syst - 2020 - Delworth - SPEAR The Next Generation GFDL Modeling System for Seasonal to Multidecadal-1.pdf skipped: [33] -> J Adv Model Earth Syst - 2020 - Delworth - SPEAR The Next Generation GFDL Modeling System for Seasonal to Multidecadal-1.md
- J Adv Model Earth Syst - 2020 - Lu - GFDL s SPEAR Seasonal Prediction System Initialization and Ocean Tendency Adjustment -1.pdf skipped: [15] -> J Adv Model Earth Syst - 2020 - Lu - GFDL s SPEAR Seasonal Prediction System Initialization and Ocean Tendency Adjustment -1.md


Quick sanity check (look at one merged file)

In [15]:
sample = written[0][1]
print("Sample merged file:", sample)
text = sample.read_text(encoding="utf-8", errors="ignore")
print(text[-2000:])


Sample merged file: /home/zappalaj/nougat_merged_md/J Adv Model Earth Syst - 2020 - Delworth - SPEAR The Next Generation GFDL Modeling System for Seasonal to Multidecadal-1.md
ther Review , 147(9), 3409–3428. https://doi.org/10.1175/MWR ‐D‐18‐0227.1
Collins, M., Knutti, R., Arblaster, J., Dufresne, J. ‐L., Fichefet, T., Gao, X., et al. (2013). Long ‐term climate change: Projections, commit-
ments and irreversibility. In T. F. Stocker et al. (Eds.), Climate Change 2013: The Physical Science Basis. Contribution of Working Group I
to the Fifth Assessment Report of the Intergovernmental Panel on Climate Change (pp. 1029–1136). Cambridge: Cambridge University
Press.
Dee, D. P., Uppala, S. M., Simmons, A. J., Berrisford, P., Poli, P., Kobayashi, S., et al. (2011). The ERA‐Interim reanalysis: Conﬁguration and
performance of the data assimilation system. Quarterly Journal of the Royal Meteorological Society , 137(656), 553–597. https://doi.org/
10.1002/qj.828
Delworth, T. L., Broccoli, A. J., 

Create Doc-level hashes (Doc-level hash (e.g., SHA256 of the PDF bytes) stored as metadata: source_hash = sha256(pdf_bytes)) per chunk to prevent uploading duplicate documents to the Chroma Vector DB

In [16]:
import json
import hashlib
from pathlib import Path

def file_sha256(path: Path, chunk_bytes: int = 1024 * 1024) -> str:
    """Fast-ish SHA256 of a file on disk."""
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for block in iter(lambda: f.read(chunk_bytes), b""):
            h.update(block)
    return h.hexdigest()

# Build a stable hash per *PDF* (useful for de-dup + stable IDs later)
pdf_hashes = {pdf.name: file_sha256(pdf) for pdf in pdfs}

out_path = MERGED_OUT / "pdf_sha256.json"
out_path.write_text(json.dumps(pdf_hashes, indent=2), encoding="utf-8")

print(f"✅ Wrote {len(pdf_hashes)} PDF hashes -> {out_path}")


### Ingest merged Markdown into Chroma

The next cell is the **single source of truth** for ingestion (no duplicate helper cells elsewhere).
Edit the configuration block at the top (`MERGED_MD_DIR`, `CHROMA_DIR`, model name, chunk sizes) and run it.


## Environment 2: `rag` (LangChain + Chroma ingestion)

Activate your RAG environment (example):

```bash
conda activate rag
```

This section will:
- load merged Markdown from `MERGED_OUT`
- sanitize/flatten metadata so Chroma won’t error
- chunk documents and embed them with `sentence-transformers`
- build (or update) a local Chroma DB on disk

If you run into dependency conflicts between Nougat and `sentence-transformers`, that’s expected — keep them in separate envs as you’ve been doing.


The cell below:
Fixes the “metadata value got [33] which is a list” class of errors permanently.

Lets you drop boilerplate (“fallback extracted text…”) that was polluting your top results.

Gives you stable IDs so reruns “upsert” instead of duplicating.

In [1]:
from __future__ import annotations

import json
import re
import hashlib
from pathlib import Path
from typing import Dict, Any, List, Tuple, Optional

from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

from langchain_community.vectorstores.utils import filter_complex_metadata

# Recommended: disable Chroma telemetry to avoid OpenTelemetry/protobuf dependency issues
import chromadb
from chromadb.config import Settings

# -------------------------
# CONFIG
# -------------------------
MERGED_MD_DIR = Path("/home/zappalaj/nougat_merged_md")   # <-- where your merged .md/.json live
VECTOR_DB_DIR = Path("/home/zappalaj/chroma_db")         # <-- where you want chroma to persist
COLLECTION    = "nougat_merged"

CHUNK_SIZE    = 1200
CHUNK_OVERLAP = 150

EMBED_MODEL   = "sentence-transformers/all-MiniLM-L6-v2"

# Keep fallback chunks, but sanitize boilerplate
SANITIZE_NUGAT_FALLBACK_HEADER = True
INFER_PAGE_FROM_TEXT = True

# -------------------------
# Helpers
# -------------------------
def sha256_text(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8", errors="ignore")).hexdigest()

def load_md_and_optional_json(md_path: Path) -> Tuple[str, Dict[str, Any]]:
    text = md_path.read_text(encoding="utf-8", errors="ignore")

    meta: Dict[str, Any] = {}
    json_path = md_path.with_suffix(".json")
    if json_path.exists():
        try:
            meta = json.loads(json_path.read_text(encoding="utf-8", errors="ignore"))
            if not isinstance(meta, dict):
                meta = {"raw_meta": meta}
        except Exception as e:
            meta = {"json_parse_error": str(e)}
    return text, meta

_PAGE_RE = re.compile(r"###\s*Page\s+(\d+)", re.IGNORECASE)
def infer_page_from_markdown(text: str) -> Optional[int]:
    m = _PAGE_RE.search((text or "")[:2000])
    return int(m.group(1)) if m else None

def normalize_page_value(v: Any) -> Optional[int]:
    if v is None:
        return None
    if isinstance(v, int):
        return v
    if isinstance(v, str) and v.strip().isdigit():
        return int(v.strip())
    if isinstance(v, list) and len(v) > 0:
        first = v[0]
        if isinstance(first, int):
            return first
        if isinstance(first, str) and first.strip().isdigit():
            return int(first.strip())
    return None

# Detect Nougat fallback boilerplate; rewrite/strip it
_FALLBACK_PHRASE_RE = re.compile(
    r"##\s*Fallback extracted text for skipped pages",
    re.IGNORECASE,
)

def sanitize_fallback_header(text: str) -> str:
    """
    Remove/replace Nougat fallback boilerplate even when it appears inline.
    Keeps the rest of the content (e.g., the Page headings and text).
    """
    if not text:
        return text

    # Remove the phrase anywhere (standalone or inline)
    cleaned = re.sub(_FALLBACK_PHRASE_RE, "", text)

    # Also remove any leftover repeated separators caused by removal
    cleaned = cleaned.replace("---", " ")

    # Normalize whitespace (important when the phrase was inline)
    cleaned = re.sub(r"[ \t]+", " ", cleaned)
    cleaned = re.sub(r"\n{3,}", "\n\n", cleaned)

    return cleaned.strip()


def normalize_source_metadata(md_path: Path, meta: Dict[str, Any]) -> Dict[str, Any]:
    out: Dict[str, Any] = {}
    out["source_path"] = str(md_path)
    out["source_name"] = md_path.name

    for key in ["title", "doc_title", "filename", "paper_title"]:
        if key in meta and isinstance(meta[key], str) and meta[key].strip():
            out["title"] = meta[key].strip()
            break

    for key in ["page", "page_num", "page_number", "pages"]:
        if key in meta:
            p = normalize_page_value(meta[key])
            if p is not None:
                out["page"] = p
            break

    for key in ["doi", "year", "authors", "journal"]:
        if key in meta:
            v = meta[key]
            if isinstance(v, (str, int, float, bool)) or v is None:
                out[key] = v
            elif isinstance(v, list):
                out[key] = ", ".join(map(str, v))
            elif isinstance(v, dict):
                out[key] = json.dumps(v, ensure_ascii=False)
            else:
                out[key] = str(v)

    return out

def force_metadata_primitives(meta: Dict[str, Any]) -> Dict[str, Any]:
    safe: Dict[str, Any] = {}
    for k, v in (meta or {}).items():
        if v is None or isinstance(v, (str, int, float, bool)):
            safe[k] = v
        else:
            try:
                safe[k] = json.dumps(v, ensure_ascii=False)
            except Exception:
                safe[k] = str(v)
    return safe

# -------------------------
# Embeddings
# -------------------------
embeddings = None
try:
    from langchain_community.embeddings import HuggingFaceEmbeddings

    embeddings = HuggingFaceEmbeddings(
        model_name=EMBED_MODEL,
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True},
    )
    print("✅ HuggingFaceEmbeddings OK:", EMBED_MODEL)

except Exception as e:
    print("⚠️ HuggingFaceEmbeddings failed; using fallback embedder.")
    print("   Reason:", repr(e))

    import numpy as np
    from langchain_core.embeddings import Embeddings

    class HashEmbeddings(Embeddings):
        def __init__(self, dim: int = 384):
            self.dim = dim

        def _vec(self, text: str) -> List[float]:
            h = hashlib.sha256(text.encode("utf-8", errors="ignore")).digest()
            arr = np.frombuffer(h, dtype=np.uint8).astype(np.float32)
            reps = int(np.ceil(self.dim / arr.size))
            v = np.tile(arr, reps)[: self.dim]
            norm = np.linalg.norm(v) + 1e-12
            return (v / norm).tolist()

        def embed_documents(self, texts: List[str]) -> List[List[float]]:
            return [self._vec(t) for t in texts]

        def embed_query(self, text: str) -> List[float]:
            return self._vec(text)

    embeddings = HashEmbeddings(dim=384)
    print("✅ Fallback HashEmbeddings OK (dim=384)")

# -------------------------
# 1) Load docs
# -------------------------
md_files = sorted(MERGED_MD_DIR.rglob("*.md"))
if not md_files:
    raise FileNotFoundError(f"No .md files found under {MERGED_MD_DIR}")

docs: List[Document] = []
for md_path in md_files:
    text, meta = load_md_and_optional_json(md_path)
    if not text.strip():
        continue

    base_meta = normalize_source_metadata(md_path, meta)
    base_meta = force_metadata_primitives(base_meta)
    docs.append(Document(page_content=text, metadata=base_meta))

print(f"✅ Loaded {len(docs)} markdown docs from {MERGED_MD_DIR}")

# -------------------------
# 2) Chunk
# -------------------------
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)

all_chunks: List[Document] = []
all_ids: List[str] = []

for d in docs:
    chunks = splitter.split_documents([d])
    for idx, c in enumerate(chunks):
        # Sanitize the repeated boilerplate header without dropping the chunk
        if SANITIZE_NUGAT_FALLBACK_HEADER:
            c.page_content = sanitize_fallback_header(c.page_content)

        # Infer page from text if missing
        if INFER_PAGE_FROM_TEXT and c.metadata.get("page") is None:
            p = infer_page_from_markdown(c.page_content)
            if p is not None:
                c.metadata["page"] = p

        # final metadata safety
        c.metadata = force_metadata_primitives(c.metadata)

        # stable IDs
        src_name = c.metadata.get("source_name", "unknown")
        page = c.metadata.get("page", "")
        content_hash = sha256_text(c.page_content)[:16]
        cid = f"{src_name}|p{page}|{idx}|{content_hash}"

        all_ids.append(cid)
        all_chunks.append(c)

print(f"✅ Chunked into {len(all_chunks)} total chunks.")

# -------------------------
# 3) Chroma-safe metadata
# -------------------------
all_chunks = filter_complex_metadata(all_chunks)
all_chunks = [Document(page_content=c.page_content, metadata=force_metadata_primitives(c.metadata)) for c in all_chunks]

# -------------------------
# 4) Build / open Chroma + ingest
# -------------------------
client = chromadb.PersistentClient(
    path=str(VECTOR_DB_DIR),
    settings=Settings(anonymized_telemetry=False),
)

vectordb = Chroma(
    collection_name=COLLECTION,
    persist_directory=str(VECTOR_DB_DIR),
    embedding_function=embeddings,
    client=client,
)

vectordb.add_documents(all_chunks, ids=all_ids)

print(f"✅ Added/updated {len(all_chunks)} chunks into collection='{COLLECTION}'")
print(f"   Persist dir: {VECTOR_DB_DIR.resolve()}")

# -------------------------
# 5) Quick sanity query
# -------------------------
query = "What is this document about?"
results = vectordb.similarity_search(query, k=3)

print("\n=== Sanity query results ===")
for i, r in enumerate(results, 1):
    src = r.metadata.get("source_name", r.metadata.get("source_path", "unknown"))
    title = r.metadata.get("title", "")
    page = r.metadata.get("page", None)
    print(f"\n[{i}] source={src}  title={title}  page={page}")
    print(r.page_content[:400].replace("\n", " ").strip(), "...")



/home/zappalaj/miniconda3/envs/rag_new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_18521/2725413480.py:157: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


✅ HuggingFaceEmbeddings OK: sentence-transformers/all-MiniLM-L6-v2
✅ Loaded 2 markdown docs from /home/zappalaj/nougat_merged_md
✅ Chunked into 269 total chunks.
✅ Added/updated 269 chunks into collection='nougat_merged'
   Persist dir: /home/zappalaj/chroma_db

=== Sanity query results ===

[1] source=J Adv Model Earth Syst - 2020 - Delworth - SPEAR The Next Generation GFDL Modeling System for Seasonal to Multidecadal-1.md  title=  page=33
### Page 33 ...

[2] source=J Adv Model Earth Syst - 2020 - Lu - GFDL s SPEAR Seasonal Prediction System Initialization and Ocean Tendency Adjustment -1.md  title=  page=15
### Page 15 ...

[3] source=J Adv Model Earth Syst - 2020 - Delworth - SPEAR The Next Generation GFDL Modeling System for Seasonal to Multidecadal-1.md  title=  page=None
Figure 19: Time series of an index of the Atlantic Meridional Overturning Circulation (AMOC) in the North Atlantic, computed using isopycal coordinates (see Figures 12c and 12d). The index is computed each year 